# WS04: LU Factorisation and iterative methods

These exercises are indented to give you practice at using the material on numerical approximation and are intended to reinforce the material that was covered in lectures.

Please attempt the worksheet before your tutorial. Support is available in your tutorial or in the Class Team.

*These are partial solutions. Please create [Issues](https://gitlab.com/comp2421-numerical-computation/book/-/issues) and [Merge requests](https://gitlab.com/comp2421-numerical-computation/book/-/merge_requests) with your solutions.*

## Part a (pen and paper warm up)

### 1. LU factorisation
Use LU factorisation to solve the linear system of equations given by

   $$
   \begin{pmatrix}
   4 & -1 & -1 \\ 2 & 4 & 2 \\ 1 & 2 & 4
   \end{pmatrix}
   \begin{pmatrix}
   x_1 \\ x_2 \\ x_3
   \end{pmatrix} =
   \begin{pmatrix}
   9 \\ -6 \\ 3
   \end{pmatrix}.
   $$


### 2. Jacobi iteration
Rewrite the Jacobi iteration as:
$$
\vec{x}^{(k+1)} = D^{-1} \vec{b} - D^{-1}(L+U) \vec{x}^{(k)},
$$
or
$$
\vec{x}^{(k+1)} = \vec{p} - P \vec{x}^{(k)},
$$
with $P=D^{-1}(L+U)$ and $\vec{p}=D^{-1} \vec{b}$, which facilitates computing and coding. Apply the above Jacobi iteration to the following linear system of equations. Take three iterations starting from an initial guess $(0, 0)^T$.

   $$
   \begin{pmatrix}
   2 & 1 \\ 
   1 & 2 
   \end{pmatrix}
   \begin{pmatrix}
   x_1 \\ 
   x_2 
   \end{pmatrix} =
   \begin{pmatrix}
    3 \\ 3 
   \end{pmatrix}. 
   $$
Note that the exact solution is $(1,1)^T$. If we want to reduce the error -- $ \|\text{exact solution }- \text{approximated solution}\|$, less than 0.1, how many iterations do we need? what is the corresponding residual?

### 3. Gauss-Seidel iteration
Apply Gauss-Seidel iteration to the above equations, and take three iterations and check whether it gives you more accurate approximation than Jacobi iteration -- all you need to do is to update $(x_1, x_2)$ in terms of components, i.e.: update $x_1$ first, then update $x_2$ using your new $x_1$.


## Part b (code implementations and testing)

### 4. Implement Gauss-Seidel iteration and  Jacobi iteration

Notice the only difference between Gauss-Seidel iteration and Jacobi iteration is that the former updates vector $\vec{x}$ component by component, while latter updates vector $\vec{x}$ as a whole.

In [1]:
import numpy as np

In [2]:
def Guass_Seidel_iteration(A, b, max_iteration):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    n=len(b) # dimensions of the linear system of equations
        
    for i in range(n):
        if np.abs(A[i,i])<1.e-15 :
            print('Diagonal element (%f %f)is zero!' % (i,i))
            return
        
    P=np.zeros([n,n])    # matrix P = D^{-1}(L+U)
    p=np.zeros(n)        # vector p = D^{-1} b
    for i in range(n):
        p[i]=b[i]/A[i,i] 
        for j in range(n):
             P[i,j] = A[i,j]/A[i,i]
                  
        P[i,i] = 0
        
    
    #create a new array to store the results, initialised as zero
    x = np.zeros(n)
    for it in range(max_iteration):
        for i in range(n):
            x[i]=p[i]
            for j in range(n):     
                x[i] = x[i] - P[i,j]*x[j]   
        
    return x

In [3]:
def Jacobi_iteration(A, b, max_iteration):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    n=len(b) # dimensions of the linear system of equations
        
    for i in range(n):
        if np.abs(A[i,i])<1.e-15 :
            print('Diagonal element (%f %f)is zero!' % (i,i))
            return
        
    P=np.zeros([n,n])    # matrix P = D^{-1}(L+U)
    p=np.zeros(n)        # vector p = D^{-1} b
    for i in range(n):
        p[i]=b[i]/A[i,i] 
        for j in range(n):
             P[i,j] = A[i,j]/A[i,i]
                  
        P[i,i] = 0
        
    
    #create a new array to store the results, initialised as zero
    '''
    x = np.zeros(n)
    for i in range(max_iteration):
        x = p - P.dot(x)
    '''
    x = np.zeros(n)
    for it in range(max_iteration):
        for i in range(n):
            b[i]=p[i]
            for j in range(n):     
                b[i] = b[i] - P[i,j]*x[j]        
        x=b.copy()
        
    return x

### 5. Test your iterative methods

In [4]:
# Test different linear solvers starting from the above two-dimensional linear system
A = np.array([[2, 1], [1, 2]])
b = np.array([3, 3])
x_exact = np.array([1,1])

# numpy linear solver
x0 = np.linalg.solve(A,b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 4)
print("Solution by Jacobi iteration: ",x)
print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

x = Guass_Seidel_iteration(A, b, 4)
print("Solution by Guass Seidel iteration: ",x)
print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [1. 1.]
Solution by Jacobi iteration:  [0.984375  1.0078125]
Error:  [-0.015625   0.0078125]
Residual:  [-0.0234375  0.       ]
Solution by Guass Seidel iteration:  [1.0078125  0.99609375]
Error:  [ 0.0078125  -0.00390625]
Residual:  [0.01171875 0.        ]


In [5]:
A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 100)
print("Solution by Jacobi iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)

x = Guass_Seidel_iteration(A, b, 100)
print("Solution by Guass Seidel iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Solution by Jacobi iteration:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [5.90052451e-11 3.31645822e-11 7.17648163e-12 0.00000000e+00]
Solution by Guass Seidel iteration:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [6.37108144e-11 3.58131302e-11 7.74846853e-12 0.00000000e+00]


In [6]:
n=20
B = np.random.rand(n, n)
eps = 10
A = eps * np.eye(n) + B * B.T
b = np.random.rand(n)

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 100)
print("Solution by Jacobi iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)

x = Guass_Seidel_iteration(A, b, 100)
print("Solution by Guass Seidel iteration: ",x)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [ 0.00942184  0.03138309  0.07503168  0.02636373  0.08085662  0.06848033
  0.06765081  0.02755309  0.05819485  0.01942784  0.03255753  0.05657507
  0.02496764 -0.0061455  -0.0132695   0.05705425  0.01027244  0.02095097
 -0.00075671  0.01539811]
Solution by Jacobi iteration:  [ 0.00942184  0.03138309  0.07503168  0.02636373  0.08085662  0.06848033
  0.06765081  0.02755309  0.05819485  0.01942784  0.03255753  0.05657507
  0.02496764 -0.0061455  -0.0132695   0.05705425  0.01027244  0.02095097
 -0.00075671  0.01539811]
Residual:  [ 0.00000000e+00 -5.55111512e-17 -1.11022302e-16 -5.55111512e-17
  1.11022302e-16  1.11022302e-16  0.00000000e+00  1.11022302e-16
  3.33066907e-16  5.55111512e-17 -1.66533454e-16 -1.11022302e-16
  5.55111512e-17 -2.77555756e-17  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -5.55111512e-17  0.00000000e+00  5.55111512e-17]
Solution by Guass Seidel iteration:  [ 0.00942184  0.03138309  0.07503168  0.02636373  0.08085662  0.06848033
  0.06

## Part c: Extension


### 6. Convergence
For the following iteration formulas:
$$
\vec{x}^{(k+1)} = 
\begin{pmatrix}
3  \\ 
1  
\end{pmatrix} 
-
\begin{pmatrix}
 2 & 0 \\ 
0 & 2 
\end{pmatrix} 
\vec{x}^{(k)},
$$
$$
\vec{x}^{(k+1)} = 
\begin{pmatrix}
3  \\ 
1  
\end{pmatrix} 
-
\begin{pmatrix}
1 & 0 \\ 
0 & 1 
\end{pmatrix} 
\vec{x}^{(k)}.
$$
and
$$
\vec{x}^{(k+1)} = 
\begin{pmatrix}
3  \\ 
1  
\end{pmatrix} 
-
\begin{pmatrix}
1/2 & 0 \\ 
0 & 1/2 
\end{pmatrix} 
\vec{x}^{(k)}.
$$

- Take two or three iterations from an arbitrary initial guess. Do you think which iteration can converge?
- Do you think the convergence of Jacobi iteration is completely determined by the properties of matrix $P$?
- Wikipedia: <a href="https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors">Eigenvaluesdecomposition</a> is the right tool to decompose a matrix to diagonal matrix, which allows us to analyse the convergence of an iteration method but is beyond the scope of this module.